### Import Divvy Bike Data: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-publictripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv<br>
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv<br>

### Import Weather Data: 
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline weather_daily.csv

In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [13]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [3]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['weather_daily', 'divvy_ridedata', 'divvy_ridedata_merged']


In [ ]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

In [8]:
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']

In [ ]:
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))

In [ ]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [ ]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [ ]:
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())

In [9]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])


'date_1'

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ridedata_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")


In [10]:
divvy_ridedata_merged = db['divvy_ridedata_merged']
divvy_ridedata_merged.find_one()

{'_id': ObjectId('64e4eab4a682807a5695d165'),
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-13 12:02:44',
 'ended_at_date': '2022-01-13',
 'ended_at_time': '12:02:44',
 'member_casual': 'casual',
 'ride_id': 'C2F7DD78E82EC875',
 'rideable_type': 'electric_bike',
 'start_lat': 42.0128005,
 'start_lng': -87.665906,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-13 11:59:47',
 'started_at_date': '2022-01-13',
 'started_at_time': '11:59:47',
 'weather_data': {'_id': ObjectId('64e4ebecb589d1838b565c57'),
  'date': '2022-01-13',
  'cloud_cover': 75.0,
  'precipitation': 0.0,
  'min_temp': 30.72,
  'max_temp': 39.83,
  'morning_temp': 35.49,
  'afternoon_temp': 30.72,
  'evening_temp': 34.38,
  'night_temp': 39.22,
  'max_windspeed': 8.01}}

In [15]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_ridedata', 'weather_daily', 'divvy_ridedata_merged']


In [17]:
# Use aggregation pipeline to create a collection that contains start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$ne": ""}, 
                     "end_station_name":{"$exists": True, "$ne": ""}}},

         {"$out": "withStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))


In [19]:
#check to make sure that collections is updated
db.list_collection_names()

['divvy_ridedata', 'weather_daily', 'withStationName', 'divvy_ridedata_merged']

In [20]:
#check if there are any documents without start or end station names
print(db.withStationName.find_one({"start_station_name":""}))
print(db.withStationName.find_one({"end_station_name":""}))

None
None


In [21]:
# Use aggregation pipeline to create a collection that doesn't start and end station names
pipeline = [
         {"$match": {"start_station_name": {"$exists": True, "$eq": ""}, 
                     "end_station_name":{"$exists": True, "$eq": ""}}},

         {"$out": "withoutStationName"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

In [22]:
db.list_collection_names()

['divvy_ridedata',
 'weather_daily',
 'withoutStationName',
 'withStationName',
 'divvy_ridedata_merged']

In [38]:
#check if there are any documents without start or end station names
print(db.withoutStationName.find_one({"start_station_name":{"$ne":""}}))
print(db.withoutStationName.find_one({"end_station_name":{"$ne":""}}))

None
None


In [9]:
# Use aggregation pipeline to find top ten end stations 
pipeline = [
    {
        "$group": {
            "_id": "$end_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 11
    }
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

# Print the result
for doc in result:
    print(doc)

{'_id': '', 'count': 892742, 'latitude': 41.9, 'longitude': -87.64}
{'_id': 'Streeter Dr & Grand Ave', 'count': 75382, 'latitude': 41.892278, 'longitude': -87.612043}
{'_id': 'DuSable Lake Shore Dr & North Blvd', 'count': 42141, 'latitude': 41.911722, 'longitude': -87.626804}
{'_id': 'Michigan Ave & Oak St', 'count': 40127, 'latitude': 41.90096039, 'longitude': -87.62377664}
{'_id': 'DuSable Lake Shore Dr & Monroe St', 'count': 40125, 'latitude': 41.880958, 'longitude': -87.616743}
{'_id': 'Wells St & Concord Ln', 'count': 37421, 'latitude': 41.912133, 'longitude': -87.634656}
{'_id': 'Millennium Park', 'count': 35234, 'latitude': 41.8810317, 'longitude': -87.62408432}
{'_id': 'Clark St & Elm St', 'count': 34490, 'latitude': 41.902973, 'longitude': -87.63128}
{'_id': 'Theater on the Lake', 'count': 32988, 'latitude': 41.926277, 'longitude': -87.630834}
{'_id': 'Kingsbury St & Kinzie St', 'count': 32380, 'latitude': 41.88917683258, 'longitude': -87.6385057718}
{'_id': 'Wells St & Elm St

In [ ]:
#Create a pipeline query to find the top ten bike routes (by start and end station)
pipeline = [
    {
        "$group": {
            "_id": { "Start Station": "$start_station_name", "End Station": "$end_station_name"},
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 11
    }
]
# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))
# Print the result
for doc in result:
    print(doc)

In [ ]:
#test